In [ ]:
import load_file as lf
import zip_codes
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import *
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import *
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import *
from category_encoders import *
import model_metrics as mm
from imblearn.under_sampling import RandomUnderSampler

# Load Data

In [ ]:
zc = zip_codes.ZC()
path = '/Users/evan/Documents/root/IndividualDays/'

In [ ]:
start = time.time()

df = pd.concat([lf.temp_load(path + "geo_zip.gzip"),
               lf.temp_load(path + "bid_floor.gzip"),
               #lf.temp_load(path + "creative_size.gzip"),
               lf.temp_load(path + "creative_type.gzip"),
               lf.temp_load(path + "bid_timestamp_utc.gzip"),
               lf.temp_load(path + "inventory_interstitial.gzip"),
               lf.temp_load(path + "inventory_source.gzip"),
               lf.temp_load(path + "platform_bandwidth.gzip"),
               lf.temp_load(path + "platform_carrier.gzip"),
               lf.temp_load(path + "platform_device_make.gzip"),
               lf.temp_load(path + "platform_device_screen_size.gzip"),
               lf.temp_load(path + "clicks.gzip")],axis = 1)

end = time.time()

print('time to load file: {0:3.3f} seconds'.format(end-start))

# Clean Data

### Get a single state and coordinates from zip

In [ ]:
df['state'] = zc.zip_to_state_2(df['geo_zip'])

dfST = df.query('state == "MS"')
dfST = dfST.drop(['state'],axis = 1)

dfST['latitude'] = zc.zip_to_lat_2(dfST['geo_zip'])
dfST['longitude'] = zc.zip_to_lon_2(dfST['geo_zip'])
dfST = dfST.drop(['geo_zip'], axis = 1)

In [ ]:
dfST['lat2']=dfST['latitude']**2
dfST['lon2']=dfST['longitude']**2

### Format time

In [ ]:
dfST['time'] = dfST.bid_timestamp_utc.dt.hour + \
dfST.bid_timestamp_utc.dt.minute/60 + dfST.bid_timestamp_utc.dt.second/3600
dfST['weekday'] = dfST.bid_timestamp_utc.dt.dayofweek
dfST['yearday'] = dfST.bid_timestamp_utc.dt.dayofyear
dfST = dfST.drop(['bid_timestamp_utc'],axis = 1)

In [ ]:
# dfSTtr = dfST.iloc[:450000, :]
# dfSTcv = dfST.iloc[450000:, :]
# dfST = None

In [ ]:
# dfSTtr.shape,dfSTcv.shape

In [ ]:
# dfSTtrY = dfSTtr.clicks
# dfSTtr = dfSTtr.drop(['clicks'],axis = 1)
# dfSTcvY = dfSTcv.clicks
# dfSTcv = dfSTcv.drop(['clicks'],axis = 1)

### One-hot encoding on some columns

In [ ]:
dfST = pd.concat([dfST,pd.get_dummies(dfST.creative_type,dummy_na=True)],axis=1)
dfST = dfST.drop(['creative_type'],axis = 1)
dfST.columns = dfST.columns.fillna('crtvtypNaN')

In [ ]:
dfST = pd.concat([dfST,pd.get_dummies(dfST.inventory_source,dummy_na=True)],axis=1)
dfST = dfST.drop(['inventory_source'],axis = 1)
dfST.columns = dfST.columns.fillna('invsrcNaN')

In [ ]:
dfST.platform_bandwidth.unique()# I don't want to one-hot encode this one

In [ ]:
dfST = pd.concat([dfST,pd.get_dummies(dfST.platform_carrier,dummy_na=True)],axis=1)
dfST = dfST.drop(['platform_carrier'],axis = 1)
dfST.columns = dfST.columns.fillna('platcarrNaN')

In [ ]:
dfST.platform_device_make.unique()# This one should maybe not be one-hot encoded

In [ ]:
dfST.platform_device_screen_size.unique()# I don't want to one-hot encode this one

### Separate out clicks from data

In [ ]:
clicks = dfST['clicks']
dfST = dfST.drop(['clicks'],axis = 1)

### LeaveOneOut encoding for some columns

In [ ]:
enc = LeaveOneOutEncoder(cols=['platform_bandwidth', 'platform_device_make','platform_device_screen_size']).fit(dfST,clicks)
dfSTnum = enc.transform(dfST)

### Separate into training and cv sets

Since data is time ordered, splitting the data this way makes sense

In [ ]:
dfSTtr = dfSTnum.iloc[:450000, :]
dfSTcv = dfSTnum.iloc[450000:, :]
clickstr = clicks.iloc[:450000]
clickscv = clicks.iloc[450000:]
print(dfSTtr.shape,dfSTcv.shape,clickstr.shape,clickscv.shape)

### Feature scaling

Only scaling based on the training data

In [ ]:
scaler = StandardScaler().fit(dfSTtr)
dfSTsc_tr = pd.DataFrame(scaler.transform(dfSTtr))
dfSTsc_cv = pd.DataFrame(scaler.transform(dfSTcv))

### Make a sparse DataFrame from training data

In [ ]:
dfSTsprstr = dfSTsc_tr.to_sparse()
dfSTsprscv = dfSTsc_cv.to_sparse()
dfSTsprstr.shape,dfSTsprscv.shape

### Classifying

In [ ]:
dfSTUStr, clickUStr = RandomUnderSampler(random_state=1).fit_sample(dfSTsprstr,clickstr)
dfSTUStr.shape, clickUStr.shape

In [ ]:
s= time.time()
#clf = MLPClassifier(hidden_layer_sizes = (100,100),tol = 1e-5,warm_start = True)
clf = LinearDiscriminantAnalysis()

clf.fit(dfSTUStr, clickUStr)
print (time.time()-s)

ypred = clf.predict(dfSTsprscv)

conf,ax=plt.subplots()
mm.plot_confusion_matrix(clickscv,ypred,ax,title = "Confusion")

In [ ]:
# from https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


digits = load_digits()
X, y = digits.data, digits.target


title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=4)

plt.show();

In [ ]:
plot_learning_curve(clf,'My Learning Curve',dfSTUStr, clickUStr)